In [1]:
import pandas as pd
import datetime, re, os
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
from collections import Counter

In [2]:
file = '../../../../Public/nk-newspapers-shared/factiva-full-data.csv'
df = pd.read_csv(file)

In [3]:
df.columns

Index(['source_name', 'title', 'body', 'byline', 'publication_datetime',
       'modification_datetime', 'subject_codes', 'company_codes',
       'industry_codes', 'region_codes', 'pub_name', 'pub_date', 'part'],
      dtype='object')

In [4]:
df['pub_name'].unique()

array(['MINCHO', 'RODONG'], dtype=object)

In [5]:
df['publication_date'] = df['publication_datetime'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)/1000))

In [6]:
df_test = df[(df['publication_date']>='2023-08-01 00:00:00') & (df['pub_name']=='RODONG')]

In [7]:
def find_matches(row):
    try:
        part1 = re.sub('\s+', '', row['title'])
        part1 = [*part1]
    except:
        part1 = []
    try:
        part2 = re.sub('\s+', '', row['body'])
        part2 = [*part2]
    except:
        part2 = []
    small_list = part1 + part2
    
    ocr_folder = '../../../../Public/OCR_test_files/rd_' + row['publication_date'].strftime('%Y-%m-%d') + '_ocr/'
    li_ocr_pages = [file for file in sorted(os.listdir(ocr_folder)) if file[-3:] == 'txt']
    
    matching = {}
    for ocr_page in li_ocr_pages:
        with open(ocr_folder+ocr_page, encoding='utf8') as f:
            lines = f.readlines()
        string = re.sub('\s+', '', ''.join(lines))
        big_list = [*string]
        overlap = list((Counter(big_list) & Counter(small_list)).elements())
        ratio = len(overlap)/len(small_list)    
        matching[ocr_page] = ratio
    sorted_ratios = sorted(matching.items(), key=lambda x:x[1], reverse=True)

    row['1st_best_page'],  row['1st_best_ratio'] = sorted_ratios[0]
    row['2nd_best_page'],  row['2nd_best_ratio'] = sorted_ratios[1]
    row['3rd_best_page'],  row['3rd_best_ratio'] = sorted_ratios[2]
    
    return row

In [8]:
df_test = df_test.apply(find_matches, axis=1)

In [9]:
df_test[[
    'publication_date', 'title', 'body',
    '1st_best_page', '1st_best_ratio', '2nd_best_page', '2nd_best_ratio', '3rd_best_page', '3rd_best_ratio'
]].head(10)

,publication_date,title,body,1st_best_page,1st_best_ratio,2nd_best_page,2nd_best_ratio,3rd_best_page,3rd_best_ratio
11994,2023-08-01,<재해성기후에 대처하기 위한 사업을 계속 강하게 내밀자> 농작물보호대책을 철저히 세워,물론 농장에서는 큰물피해를 최소화하는데 큰 의의를 부여하고 이미전부터 이와 관련한 ...,rd_2023-08-01_url_5_p5.txt,0.971084,rd_2023-08-01_url_5_p6.txt,0.921687,rd_2023-08-01_url_5_p3.txt,0.918072
11995,2023-08-01,생산실적과 공급실적이 같은가,"지난 ７월에도 북창지구청년탄광련합기업소, 득장지구탄광련합기업소를 비롯한 여러 련합기...",rd_2023-08-01_url_5_p5.txt,0.973487,rd_2023-08-01_url_5_p3.txt,0.884726,rd_2023-08-01_url_5_p6.txt,0.873775
11996,2023-08-01,한가정의 어머니처럼,소대원들의 아픔을 자기 아픔으로 여기고 따뜻이 돌봐주고 생활상애로가 제기될 때마다 ...,rd_2023-08-01_url_5_p3.txt,0.983333,rd_2023-08-01_url_5_p6.txt,0.963333,rd_2023-08-01_url_5_p4.txt,0.960000
11997,2023-08-01,<서포지구 새 거리건설을 제기일에 최상의 수준으로 완공할 기세드높다> 뜨거운 애국의...,당의 구상과 의도를 높이 받들고 수도 평양에 웅장화려한 새 거리를 일떠세우는 보람찬...,rd_2023-08-01_url_5_p1.txt,0.971226,rd_2023-08-01_url_5_p2.txt,0.886676,rd_2023-08-01_url_5_p6.txt,0.881806
11998,2023-08-01,괴뢰지역에서 악성비루스감염자 급속히 증가,지난 １９일 하루동안에만도 ４만 ７ ０２９명이 악성비루스에 감염되였다.\n\n괴뢰지...,rd_2023-08-01_url_5_p6.txt,0.909091,rd_2023-08-01_url_5_p4.txt,0.818182,rd_2023-08-01_url_5_p1.txt,0.808081
11999,2023-08-01,이스라엘의 도발적인 행위 비난,"알 아크사사원은 이슬람교의 성지라고 하면서 그는 꾸드스의 력사적, 합법적지위를 허물...",rd_2023-08-01_url_5_p6.txt,0.976744,rd_2023-08-01_url_5_p2.txt,0.930233,rd_2023-08-01_url_5_p1.txt,0.906977
12000,2023-08-01,재해성기후에 대처하기 위한 사업을 계속 강하게 내밀자,NaN,rd_2023-08-01_url_5_p1.txt,1.000000,rd_2023-08-01_url_5_p2.txt,1.000000,rd_2023-08-01_url_5_p3.txt,1.000000
12001,2023-08-02,진심은 사람을 움직이는 가장 큰 힘이다,그것은 바로 사람들을 진심으로 뜨겁게 대하는것이다.\n\n경애하는 김정은동지께서는 ...,rd_2023-08-02_url_5_p3.txt,0.964399,rd_2023-08-02_url_5_p2.txt,0.923077,rd_2023-08-02_url_5_p4.txt,0.910998
12002,2023-08-02,<당세포들이 틀어쥐고나가야 할 중요과업> 대중을 당의 로선과 정책으로 튼튼히 무장시...,경애하는 김정은동지께서는 다음과 같이 말씀하시였다.\n\n《기층조직을 강화하여 전당...,rd_2023-08-02_url_5_p3.txt,0.974576,rd_2023-08-02_url_5_p2.txt,0.952542,rd_2023-08-02_url_5_p5.txt,0.951695
12003,2023-08-01,대를 이어 하는것이 참된 애국이다,우리 나라에는 아버지가 섰던 초소를 아들이 지키고 어머니가 맡았던 기대를 딸이 돌리...,rd_2023-08-01_url_5_p4.txt,0.966276,rd_2023-08-01_url_5_p2.txt,0.928886,rd_2023-08-01_url_5_p6.txt,0.923021
